# A (Possible) Code for Matching Timepix Clusters to Tagger Hits

First, we'll import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import Timepix
import ROOT
import numpy as np
import root_numpy
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed and the tagger calibration file (?):

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

aqdet.LoadDetectors(["taggerWithOffsets.json"])

Mk2 Data
taggerWithOffsets.json


In [ ]:
triggerTime  = []
scalerRepeat = 0
prevLong     = 0
pulseCount   = 0
EPICSN       = 0
EPICSEventA  = 0
EPICSEventB  = 0

timeWindowMin = -400000
timeWindowMax = 400000

timeMax = 200000000
nBins = 300


timeWin       = 2000000
timeDiffDouble      = Hist(nBins,-timeWin,timeWin,name='DoubleCoincidence')
timeDiff      = Hist(nBins,-timeWin,timeWin,name='Coincidence')

timeA         = Hist(40000,0,timeMax,name='TimeA')
timeB         = Hist(40000,0,timeMax,name='TimeB')

#h = Hist2D((timeWindowMax-timeWindowMin)/1000,timeWindowMin,timeWindowMax,40,0,40)
hA  = Hist((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,name='TimesA')
hA2  = Hist((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,name='TimesA2')
h2A = Hist2D((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,100,0,timeMax,name='TimesA2D')
hB  = Hist((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,name='TimesB')
h2B = Hist2D((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,100,0,timeMax,name='TimesB2D')

prevTriggerTime = [0]

positionA = Hist2D(256,0,256,256,0,256,name='posA')
positionACoin = Hist2D(256,0,256,256,0,256,name='posACoin')

def TimepixTagger():
    global prevTriggerTime
    global triggerTime
    global scalerRepeat
    global prevLong
    global pulseCount
    global EPICSEventA
    global EPICSEventB
    global EPICSN
    
    
    ## why 104,102,103? Is it like a reference number?
    ADC104    = aq.adcArray[np.where(aq.adcArray['adc']==104)[0]]['val'][0]     
    shortTime = aq.adcArray[np.where(aq.adcArray['adc']==102)[0]]['val'][0]
    longTime  = aq.adcArray[np.where(aq.adcArray['adc']==103)[0]]['val'][0]
    
    ## no idea what this does or why
    if(longTime<prevLong): 
        scalerRepeat += 1
    
    ## I guess this is just how you calculate trigger time?
    triggerTime += [8*(shortTime+65536*(longTime+65536*scalerRepeat))]
    
    ## if the trigger time is too low, just stop
    if(len(triggerTime)<=1): 
        return
    
    ## now we go through the buffers like before with the timepix
    if(aq.epicsEvent==1):
        ## don't repeat events
        if(EPICSEventA!=aq.getEpicsPV('PPOL:TIMEPIXA:EVENT') and EPICSEventB!=aq.getEpicsPV('PPOL:TIMEPIXB:EVENT') ):
            ## get the data from the timepix?
            EPICSEventA  = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
            EPICSEventB  = aq.getEpicsPV('PPOL:TIMEPIXB:EVENT')
            nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
            encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
            encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
            
            ## these are arrays of data, like before
            TimepixAData = Timepix.Decode(nHitsA,encodedA)
            TimepixBData = Timepix.Decode(nHitsB,encodedB)
            
            ## this gives us time in units of 1.65 ns ?
            nsTimeA = 16*TimepixAData[['ToA']].astype(long) - TimepixAData[['FToA']].astype(long)
            nsTimeB = 16*TimepixBData[['ToA']].astype(long) - TimepixBData[['FToA']].astype(long)
            
            ## a list containing 0, and then the time differences between hits (should it not be sorted first?)
            diffA = np.concatenate([[0],np.diff(nsTimeA)])
            diffB = np.concatenate([[0],np.diff(nsTimeB)]) 
            print(diffA)
            
            ## a list containing 0, the indices where diffA>200, and the number of time points we have
            breakA = np.concatenate([[0],np.where(diffA>200)[0],[len(nsTimeA)]])
            breakB = np.concatenate([[0],np.where(diffB>200)[0],[len(nsTimeB)]])
            ## is it like a place where we break up the clusters maybe???
            
            
            timesA = []
            timesB = []
            lengA  = []
            lengB  = []
            posA   = []
            posB   = []
            
            ## if we have no time points, just make break array empty
            if(len(nsTimeA)==0): 
                breakA = []
            if(len(nsTimeB)==0): 
                breakB = []
                
            ## for pairs of an element in BreakA and the proceeding element
            for ssi in zip(breakA[:-1],breakA[1:]):
                ## list of differences between each of these break times and the first break time
                diffs = nsTimeA[ssi[0]:ssi[1]]-nsTimeA[ssi[0]]
                ## difference between positions of the breaks (how many hits apart things happen?)
                leng  = ssi[1]-ssi[0]
                ## makes a matrix of the diffs and a row of size leng filled with leng, and takes the transpose  
                comb  = np.vstack([diffs,np.full(leng,leng)]).T
                ## append leng to the lengths 
                lengA  += [leng]
                ## append the time at the first break point to the times (like ToA of the cluster??)
                timesA += [nsTimeA[ssi[0]]]
                ## append the position at the first break point to the positions
                posA   += [TimepixAData[['x','y']][ssi[0]]]
            for ssi in zip(breakB[:-1],breakB[1:]):
                diffs = nsTimeB[ssi[0]:ssi[1]]-nsTimeB[ssi[0]]
                leng  = ssi[1]-ssi[0]
                comb  = np.vstack([diffs,np.full(leng,leng)]).T
                lengB  += [leng]
                timesB += [nsTimeB[ssi[0]]]
                posB   += [TimepixBData[['x','y']][ssi[0]]]
            
            
            ## turn these lists into numpy arrays
            timesA = np.array(timesA)
            timesB = np.array(timesB)
            posA = np.array(posA)
            posB = np.array(posB)
            
            ## fill some histograms with positions and times
            positionA.fill_array(np.vstack([posA['x'],posA['y']]).T)
            timeA.fill_array(timesA)
            timeB.fill_array(timesB)
                        
                
                
                
            ## why increment the epics number??    
            EPICSN += 1
            
            ## make an array of the trigger times?? Is this a timepix trigger?
            triggerTime2 = np.array(triggerTime,dtype=np.float)
            ## get the first index where the difference between trigger times is negative (i.e. it goes bad?)
            triggerWrap  = np.where(np.diff(triggerTime2)<0)[0]
            ## if there ARE trigger times that go negative (basically)
            if(len(triggerWrap)>0):
                ## get all the trigger times past that bad point, and fix/change them??? why these numbers?
                triggerTime2[triggerWrap[0]:] += 65536*65536*8
            
            
            ## this bit I have no idea why or what it is
            ## get the last trigger time
            timeSub = prevTriggerTime[-1]
            ## if the diff between the last trigger time and the previous trigger time is small AND there is a previous trigger time
            if((triggerTime2-timeSub)[-1]<1000000000 and len(prevTriggerTime)>1): 
                ## then change timeSub to the second last previous trigger time???
                timeSub = prevTriggerTime[-2]
    
            
            ## subtract this time sub from each trigger time
            triggerTime2 -= timeSub#triggerTime[0]
            ## and divide the trigger time by this number???
            triggerTime2 /= 1.015595*1.5625

            
            coinIdx = []
            
            ## for index (i) and time (hit) in the array of first break point times
            for i, hit in enumerate(timesA):
                ## get the difference between the trigger time and hit time
                tdiff = triggerTime2-hit
                ## if (holy moly what is this) -> if the length of the time difference at the indices less than 10000 is greater than zero or ...
                ## if you have tdiffs???
                if (len(tdiff[np.abs(tdiff)<10000])>0 or len(tdiff[np.abs(tdiff-263000)<10000])>0):
                    ## coincidence index is here, but why??
                    coinIdx +=  [i] 
            ## not sure why this couldn't just be the same for loop
            for i, hit in enumerate(timesA):
                ## get this time diff again
                tdiff = triggerTime2-hit
                ## fill some histograms
                h2A.fill_array(np.vstack((tdiff,np.full(len(triggerTime2),hit))).T)
                hA.fill_array(tdiff)
                hA2.fill_array(tdiff-263000)
                
            ## get the time and position at the coincidence and make a histogram     
            coinHit  = timesA[coinIdx]
            coinPosA = posA[coinIdx]
            positionACoin.fill_array(np.vstack([coinPosA['x'],coinPosA['y']]).T)
            
            ## for each hit in the times for B
            for hit in timesB:
                ## tdiff is the difference between A trigger time and B time?
                tdiff = triggerTime2-hit 
                ## fill histograms
                h2B.fill_array(np.vstack((tdiff,np.full(len(triggerTime2),hit))).T)
                hB.fill_array(tdiff)
                ## differences between the coincidence time at A and B time, and the time at A and B time, fill histograms
                tpxDiff  = coinHit-hit
                tpxDiff2 = timesA-hit
                timeDiff.fill_array(tpxDiff2)
                timeDiffDouble.fill_array(tpxDiff)
                
                
            ## god knows why
            triggerfilter = prevTriggerTime[-1]
            if(triggerTime[0]>triggerfilter): 
                triggerfilter += 65536*65536*8
            triggerTime     = triggerTime[np.where(triggerTime<=triggerfilter)[0][-1]:]
            
            ## reset the previous trigger time
            prevTriggerTime = []
            
            
    if(aq.eventNo%10000==0):
        print("Events Processed: ",aq.eventNo)
    ## what is the pulse count???
    if(ADC104!=pulseCount):
        pulseCount  = ADC104        
        ## set previous trigger time to the last trigger time we had
        prevTriggerTime += [triggerTime[-1]]
  
aq.runFunction(TimepixTagger,0,500000)



In [ ]:
taggerChannels = aqdet.Channels["Tagger"]     # number of channels in the tagger (368)
TpxEvent = 0

def plotCalTagger():
    global TpxEvent
    data = aqdet.Calibrate(aq.adcArray)              # calibrate the data
    taggerTimes = aqdet.Arrays["Tagger"]["Time"]     # get the tagger times

    if(aq.epicsEvent==1):        #if this event contained and EPICS buffer keep going
        if(aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')!=TpxEvent): #if the EPICS buffer has Timepix data
            
            nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')    #get the epics buffer info
            encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
            encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
            
            eventA = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
            
            TimepixA = Timepix.Decode(nHitsA,encodedA)


            for dat in taggerTimes:                          # for each data point
                for time in dat["Time"]:                     # for each tagger time
                    channel = dat["channel"]                 # get the current channel        
            print(channel, time, eventA, len(TimepixA['ToT']))
            
            TpxEvent = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
    
        
                    
    if(aq.eventNo%5000==0):                          # print a processing statement every 5000 events
        print("Events Processed: "+str(aq.eventNo))
    
aq.runFunction(plotCalTagger,0,2)               # run this process over 500000 events

In [ ]:
def TestTimings():
    
    print(aq.adcArray['adc'])
    
aq.runFunction(TestTimings,0,50000)